In [343]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import numpy as np
import datetime

In [344]:
from models import Candle, MarketTrade
from database import Database, MyWoWDatabase
import services.coinbase_services as cb
from coinbase.rest import RESTClient

In [345]:
client = cb.get_client()
db = Database('mywow.db')
dbms = MyWoWDatabase('mywow.db')

trading_pair = 'SWELL-USD'
start = datetime.datetime(year=2025, month=2, day=10, hour=1, minute=30, second=0, tzinfo=cb.LOCAL_TZ)
end = datetime.datetime(year=2025, month=2, day=10, hour=3, minute=00, second=0, tzinfo=cb.LOCAL_TZ)

In [346]:
# Assuming at root of project directory: "C:\...\MyWoW\"
import os
data_dir_path = os.path.join(os.getcwd(), 'data')
if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)

analysis_dir_path = os.path.join(data_dir_path, 'analysis')
if not os.path.exists(analysis_dir_path):
    os.mkdir(analysis_dir_path)

DATA_FETCHED = False
history_header = ','.join(['trading_pair', 'start_date', 'end_date', 'created_date', 'last_updated']) + '\n'
analysis_history_file = os.path.join(analysis_dir_path, 'history.csv')
if not os.path.exists(analysis_history_file):
    with open(analysis_history_file, 'x') as f:
        f.write(history_header)
        f.write(','.join([trading_pair, start.isoformat(), end.isoformat(), datetime.datetime.now(tz=cb.LOCAL_TZ).isoformat(), datetime.datetime.now(tz=cb.LOCAL_TZ).isoformat()]) + '\n')
else:
    with open(analysis_history_file, 'r') as f:
        header = f.readline()
        for line in f.readlines():
            prev_trading_pair, prev_start_date, prev_end_date, _, _ = line.strip('\n').split(',')
            if prev_trading_pair == trading_pair and prev_start_date == start.isoformat() and prev_end_date == end.isoformat():
                DATA_FETCHED = True

In [347]:
def fetch_and_upload_data(client: RESTClient, trading_pair: str, start_time: datetime.datetime, end_time: datetime.datetime):
    # fetch all market trades between time range and upload to db
    market_trades = cb.fetch_market_trades(client, trading_pair, start_time, end_time, cb.CANDLES_LIMIT_MAX)
    for market_trade_data in market_trades:
        market_trade = MarketTrade(market_trade_data)
        dbms.add_item(table_name='market_trade', values=market_trade.get_values())
    
    # fetch candles between same time range as market trades and upload to db
    candles = cb.fetch_market_trade_candles(client, trading_pair, start_time, end_time, cb.CANDLES_LIMIT_MAX)
    for candle_data in candles:
        candle = Candle(candle_data)
        dbms.add_item(table_name='market_candles', values=candle.get_values(market_trade_candle=True))

def reset_fetch_data(trading_pair: str, start_time: datetime.datetime, end_time: datetime.datetime):
    res = dbms.get_items(
        table_name='market_trade', 
        where_statement=f"WHERE trading_pair='{trading_pair}' AND time BETWEEN '{start_time.isoformat()}' AND '{end_time.isoformat()}'"
        )
    market_trades = [MarketTrade(data) for data in res] 
    for market_trade in market_trades:
        dbms.remove_item(table_name='market_trade', values={'trade_id':market_trade.trade_id})

    res = dbms.get_items(
        table_name='market_candles',
        where_statement=f"WHERE trading_pair='{trading_pair}' AND time BETWEEN '{start_time.isoformat()}' AND '{end_time.isoformat()}'"
    )
    candles = [Candle(data) for data in res]
    for candle in candles:
        dbms.remove_item(table_name='market_candles', values={'candle_id':candle.candle_id})

In [348]:
# reset_fetch_data(trading_pair=trading_pair, start_time=start, end_time=end)
if not DATA_FETCHED:
    fetch_and_upload_data(trading_pair=trading_pair, start_time=start, end_time=end)

In [349]:
res = dbms.get_items( 
    table_name='market_candles', 
    where_statement=f"WHERE trading_pair='{trading_pair}' AND time BETWEEN '{start.isoformat()}' AND '{end.isoformat()}'")
market_candles = [Candle(candle) for candle in res]

df_candles = pd.DataFrame(data=[candle.to_dict() for candle in market_candles])
df_candles['timestamp'] = df_candles['date'].transform(lambda x: x.timestamp())
df_candles['time'] = df_candles['date'].transform(lambda x: x.strftime('%H:%M:%S'))
df_candles['hour'] = df_candles['date'].transform(lambda x: x.hour)
df_candles['minute'] = df_candles['date'].transform(lambda x: x.minute)
df_candles.sort_values(by='date', inplace=True)
df_candles.dtypes
df_candles.head()

,candle_id,date,start,trading_pair,open,high,low,close,volume,timestamp,time,hour,minute
66,SWELL-1739179800,2025-02-10 01:30:00-08:00,1739179800,SWELL-USD,0.01171,0.01172,0.01168,0.01172,12935.0,1.739180e+09,01:30:00,1,30
65,SWELL-1739179980,2025-02-10 01:33:00-08:00,1739179980,SWELL-USD,0.01173,0.01178,0.01172,0.01176,94445.0,1.739180e+09,01:33:00,1,33
64,SWELL-1739180220,2025-02-10 01:37:00-08:00,1739180220,SWELL-USD,0.01166,0.01166,0.01164,0.01164,149042.0,1.739180e+09,01:37:00,1,37
63,SWELL-1739180760,2025-02-10 01:46:00-08:00,1739180760,SWELL-USD,0.01173,0.01186,0.01173,0.01180,94002.0,1.739181e+09,01:46:00,1,46
62,SWELL-1739181060,2025-02-10 01:51:00-08:00,1739181060,SWELL-USD,0.01176,0.01176,0.01173,0.01176,43982.0,1.739181e+09,01:51:00,1,51


In [351]:
res = dbms.get_items(
    table_name='market_trade',
    where_statement=f"WHERE trading_pair='{trading_pair}' AND time >= '{start.isoformat()}'")
market_trades = [MarketTrade(trade) for trade in res]
market_trade_data = []
for market_trade in market_trades:
    data = market_trade.to_dict()
    data['total'] = market_trade.total
    market_trade_data.append(data)

df_trades = pd.DataFrame(data=market_trade_data)
df_trades['date'] = df_trades['time']
df_trades['timestamp'] = df_trades['date'].transform(lambda x: x.timestamp())
df_trades['time'] = df_trades['date'].transform(lambda x: x.strftime('%H:%M:%S'))
df_trades['hour'] = df_trades['date'].transform(lambda x: x.hour)
df_trades['minute'] = df_trades['date'].transform(lambda x: x.minute)
df_trades['second'] = df_trades['date'].transform(lambda x: x.second)
df_trades.head()

,trade_id,trading_pair,price,size,time,side,bid,ask,exchange,total,date,timestamp,hour,minute,second
0,1135065,SWELL-USD,0.01194,6678.0,01:58:25,BUY,0.0,0.0,UKNOWN_EXCHANGE,79.73532,2025-02-10 01:58:25.908019-08:00,1.739182e+09,1,58,25
1,1135064,SWELL-USD,0.01200,9414.0,01:58:25,BUY,0.0,0.0,UKNOWN_EXCHANGE,112.96800,2025-02-10 01:58:25.737970-08:00,1.739182e+09,1,58,25
2,1135062,SWELL-USD,0.01203,9414.0,01:58:25,BUY,0.0,0.0,UKNOWN_EXCHANGE,113.25042,2025-02-10 01:58:25.732422-08:00,1.739182e+09,1,58,25
3,1135063,SWELL-USD,0.01200,12552.0,01:58:25,BUY,0.0,0.0,UKNOWN_EXCHANGE,150.62400,2025-02-10 01:58:25.732422-08:00,1.739182e+09,1,58,25
4,1135060,SWELL-USD,0.01199,767.0,01:58:25,SELL,0.0,0.0,UKNOWN_EXCHANGE,-9.19633,2025-02-10 01:58:25.647777-08:00,1.739182e+09,1,58,25


In [352]:
fig_height = 550
fig_width = 850

In [353]:
fig_pricechange = go.Figure(
    data=[go.Candlestick(
        x=df_candles.date,
        open=df_candles.open,
        high=df_candles.high,
        low=df_candles.low,
        close=df_candles.close
)])
fig_pricechange.update_layout(
    title=f'Price Change for {trading_pair} on {start.strftime('%m/%d/%y')} between {start.strftime('%H:%M')} and {end.strftime('%H:%M')}',
    yaxis=dict(
        title=dict(text='Price')
    ),
    xaxis=dict(
        title=dict(text='Time'),
        dtick=5*60*1000.0
    ),
    height=fig_height,
    width=fig_width,
    xaxis_rangeslider_visible=False)
fig_pricechange.show()

In [354]:
df_trades['hm_time'] = df_trades['date'].transform(lambda x:datetime.time(hour=x.hour, minute=x.minute))
df_trade_counts = df_trades.groupby(['hm_time', 'side'], as_index=False)['trade_id'].count()
df_trade_counts.rename(columns={'trade_id':'counts'}, inplace=True)
df_trade_counts.head()

,hm_time,side,counts
0,01:30:00,BUY,1
1,01:30:00,SELL,1
2,01:32:00,SELL,2
3,01:33:00,SELL,4
4,01:36:00,SELL,4


In [355]:
fig_tradecounts = px.bar(df_trade_counts, x='hm_time', y='counts', color='side')
fig_tradecounts.update_layout(
    title=f'{trading_pair} Market Trade Counts on {start.strftime('%m/%d/%y')} between {start.strftime('%H:%M')} and {end.strftime('%H:%M')}',
    yaxis=dict(
        title=dict(text='Counts')
    ),
    xaxis=dict(
        title=dict(text='Time')
    ),
    height=fig_height,
    width=fig_width
)
fig_tradecounts.show()

In [356]:
df_trade_totals = df_trades.groupby(['hm_time', 'side'], as_index=False)['total'].sum()
df_trade_totals.head()

,hm_time,side,total
0,01:30:00,BUY,11.23616
1,01:30:00,SELL,-2.92750
2,01:32:00,SELL,-137.34356
3,01:33:00,SELL,-505.74156
4,01:36:00,SELL,-601.94368


In [357]:
fig_tradetotals = px.bar(
    df_trade_totals,
    x='hm_time',
    y='total',
    color='side'
)
fig_tradetotals.update_layout(
    title=f"{trading_pair} Market Trade Totals on {start.strftime('%b %d')} between {start.strftime('%H:%M')} and {end.strftime('%H:%M')}",
    xaxis=dict(
        title=dict(text='Time')
    ),
    yaxis=dict(
        title=dict(text='Total')
    ),
    height=fig_height,
    width=fig_width,
)
fig_tradetotals.show()

In [358]:
fig_pricechange.show()
fig_tradecounts.show()
fig_tradetotals.show()